In [345]:
import numpy as np
import pandas as pd

from collections import Counter as c

from ast import literal_eval as eval

import xgboost as xgb

from sklearn.model_selection import train_test_split

In [429]:
from sklearn.base import TransformerMixin, BaseEstimator

from collections import Counter , defaultdict

import pandas as pd
from pandas import Series as s , DataFrame as df
import numpy as np

import seaborn as sns
# import matplotlib.pyplot as plt

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt, rcParams as rc


%matplotlib inline
rc["figure.figsize"] = 10,6

import datetime
import datetime as dt





#Algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.naive_bayes import MultinomialNB

from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier

#preprocessing
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import train_test_split

#Metrics
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.pipeline import Pipeline

# GridSearchCV to find optimal min_samples_leaf
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


import lightgbm as lgb


In [427]:
!pip install lightgbm

     |████████████████████████████████| 2.0 MB 2.0 MB/s eta 0:00:01


In [346]:
#!pip install xgboost

In [430]:
train_df = pd.read_csv("train.csv", converters={'Product_Holding_B1':eval, 'Product_Holding_B2':eval})
test_df = pd.read_csv("test.csv", converters={'Product_Holding_B1': eval})

In [431]:
train_df.shape , test_df.shape

((37748, 9), (20327, 8))

In [432]:
type(train_df.Product_Holding_B2[0])

list

In [433]:
train_df.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2
0,CC264719,Male,41,14,0,C1,S3,[P16],[P8]
1,CC209679,Female,47,14,1,C1,S2,"[P13, P20]",[P3]
2,CC319633,Female,59,14,0,C2,S2,[P11],[P00]
3,CC231413,Female,32,16,0,C1,S2,"[P8, P13]",[P6]
4,CC259633,Male,30,15,0,C2,S3,"[P16, P17, P21]","[P8, P12]"


In [434]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37748 entries, 0 to 37747
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer_ID         37748 non-null  object
 1   Gender              37748 non-null  object
 2   Age                 37748 non-null  int64 
 3   Vintage             37748 non-null  int64 
 4   Is_Active           37748 non-null  int64 
 5   City_Category       37748 non-null  object
 6   Customer_Category   37748 non-null  object
 7   Product_Holding_B1  37748 non-null  object
 8   Product_Holding_B2  37748 non-null  object
dtypes: int64(3), object(6)
memory usage: 2.6+ MB


In [435]:
c(train_df.City_Category)

Counter({'C1': 18941, 'C2': 18807})

In [436]:
c(train_df.Is_Active)

Counter({0: 27752, 1: 9996})

In [437]:
c(train_df.Gender)

Counter({'Male': 25890, 'Female': 11858})

In [438]:
c(train_df.Customer_Category)

Counter({'S3': 17865, 'S2': 13265, 'S1': 6618})

In [439]:
train_df["Gender"].astype('category')

0          Male
1        Female
2        Female
3        Female
4          Male
          ...  
37743      Male
37744      Male
37745      Male
37746    Female
37747      Male
Name: Gender, Length: 37748, dtype: category
Categories (2, object): ['Female', 'Male']

In [440]:
train_df['is_train']=1
test_df['is_train'] = 0

In [441]:
df = pd.concat([train_df, test_df])

In [442]:
df.shape, train_df.shape, test_df.shape

((58075, 10), (37748, 10), (20327, 9))

In [443]:
df.tail()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,is_train
20322,CC303542,Female,37,11,1,C2,S3,[P16],NaN,0
20323,CC266713,Male,29,14,0,C2,S2,[P16],NaN,0
20324,CC393639,Male,26,12,0,C2,S3,[P13],NaN,0
20325,CC285013,Female,44,19,1,C1,S3,"[P8, P9, P16]",NaN,0
20326,CC274657,Female,25,21,0,C1,S3,"[P12, P13, P17]",NaN,0


In [444]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58075 entries, 0 to 20326
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer_ID         58075 non-null  object
 1   Gender              58075 non-null  object
 2   Age                 58075 non-null  int64 
 3   Vintage             58075 non-null  int64 
 4   Is_Active           58075 non-null  int64 
 5   City_Category       58075 non-null  object
 6   Customer_Category   58075 non-null  object
 7   Product_Holding_B1  58075 non-null  object
 8   Product_Holding_B2  37748 non-null  object
 9   is_train            58075 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 4.9+ MB


In [445]:
df['Gender']= df.Gender.map({'Male':1,'Female':0})
df['City_Category']=df.City_Category.map({'C1':1,'C2':2,'C3':3})
df['Customer_Category'] = df.Customer_Category.map({'S1':1,'S2':2,'S3':3})




# df['Age_bin'] = pd.cut(df['Age'], [0, 20, 30, 40, 50, 60, 70, 80, 90, 100],  labels=['0-20', '20-30', '30-40', '40-50','50-60','60-70','70-80', '80-90','90-100'])




In [446]:
# df['Age_bin']

In [447]:
# df['Age_bin'] = df.Age_bin.map({'0-20':1, '20-30':2, '30-40':3, '40-50':4,'50-60':5,'60-70':6,'70-80':7, '80-90':8,'90-100':9})


# df['Age_bin']

In [448]:
# train_one_fold_x.columns

In [449]:
df.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,is_train
0,CC264719,1,41,14,0,1,3,[P16],[P8],1
1,CC209679,0,47,14,1,1,2,"[P13, P20]",[P3],1
2,CC319633,0,59,14,0,2,2,[P11],[P00],1
3,CC231413,0,32,16,0,1,2,"[P8, P13]",[P6],1
4,CC259633,1,30,15,0,2,3,"[P16, P17, P21]","[P8, P12]",1


In [450]:
def get_unique_values_from_list_col(df, col):
    local_df=df[col]
    products = []
    for item in local_df:
#         print(item)
        unique_cols = np.unique(item)
        for col in unique_cols:
            if col not in products:
                products.append(col)
    return products




def apply_transformation(row, col, prod_list, prefix):
    if row['is_train']==0 and prefix=='B2_':
        return row
    
    for item in prod_list:
        if item in row[col]:
            row[prefix+item] = 1
    return row

In [451]:
prod_list = get_unique_values_from_list_col(df, 'Product_Holding_B1')

print(prod_list)


print(len(prod_list))


b1_prod_list = ['B1_'+item for item in prod_list]
for item in b1_prod_list:
    df[item]= 0
    
df1 = df.apply(apply_transformation, col='Product_Holding_B1',prod_list=prod_list, prefix='B1_',axis=1)

['P16', 'P13', 'P20', 'P11', 'P8', 'P17', 'P21', 'P12', 'P10', 'P19', 'P2', 'P00', 'P18', 'P15', 'P6', 'P9', 'P7', 'P3', 'P5', 'P4', 'P1', 'P14']
22


In [452]:
df1.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,is_train,...,B1_P18,B1_P15,B1_P6,B1_P9,B1_P7,B1_P3,B1_P5,B1_P4,B1_P1,B1_P14
0,CC264719,1,41,14,0,1,3,[P16],[P8],1,...,0,0,0,0,0,0,0,0,0,0
1,CC209679,0,47,14,1,1,2,"[P13, P20]",[P3],1,...,0,0,0,0,0,0,0,0,0,0
2,CC319633,0,59,14,0,2,2,[P11],[P00],1,...,0,0,0,0,0,0,0,0,0,0
3,CC231413,0,32,16,0,1,2,"[P8, P13]",[P6],1,...,0,0,0,0,0,0,0,0,0,0
4,CC259633,1,30,15,0,2,3,"[P16, P17, P21]","[P8, P12]",1,...,0,0,0,0,0,0,0,0,0,0


In [453]:
c(df1.B1_P18)

Counter({0: 57542, 1: 533})

In [454]:
#df1_cols = list(df1.columns)
df1_cols = ['B1_P16',
 'B1_P13',
 'B1_P20',
 'B1_P11',
 'B1_P8',
 'B1_P17',
 'B1_P21',
 'B1_P12',
 'B1_P10',
 'B1_P19',
 'B1_P2',
 'B1_P00',
 'B1_P18',
 'B1_P15',
 'B1_P6',
 'B1_P9',
 'B1_P7',
 'B1_P3',
 'B1_P5',
 'B1_P4',
 'B1_P1',
 'B1_P14']

In [455]:
df1_cols

['B1_P16',
 'B1_P13',
 'B1_P20',
 'B1_P11',
 'B1_P8',
 'B1_P17',
 'B1_P21',
 'B1_P12',
 'B1_P10',
 'B1_P19',
 'B1_P2',
 'B1_P00',
 'B1_P18',
 'B1_P15',
 'B1_P6',
 'B1_P9',
 'B1_P7',
 'B1_P3',
 'B1_P5',
 'B1_P4',
 'B1_P1',
 'B1_P14']

In [456]:
for i in df1[df1_cols].columns:
    print(i, " : ", c(df1[i]))

B1_P16  :  Counter({0: 40748, 1: 17327})
B1_P13  :  Counter({0: 32003, 1: 26072})
B1_P20  :  Counter({0: 52374, 1: 5701})
B1_P11  :  Counter({0: 55138, 1: 2937})
B1_P8  :  Counter({0: 53925, 1: 4150})
B1_P17  :  Counter({0: 47044, 1: 11031})
B1_P21  :  Counter({0: 48258, 1: 9817})
B1_P12  :  Counter({0: 48632, 1: 9443})
B1_P10  :  Counter({0: 55829, 1: 2246})
B1_P19  :  Counter({0: 54691, 1: 3384})
B1_P2  :  Counter({0: 57075, 1: 1000})
B1_P00  :  Counter({0: 56565, 1: 1510})
B1_P18  :  Counter({0: 57542, 1: 533})
B1_P15  :  Counter({0: 57213, 1: 862})
B1_P6  :  Counter({0: 57910, 1: 165})
B1_P9  :  Counter({0: 57351, 1: 724})
B1_P7  :  Counter({0: 57482, 1: 593})
B1_P3  :  Counter({0: 57977, 1: 98})
B1_P5  :  Counter({0: 58024, 1: 51})
B1_P4  :  Counter({0: 58020, 1: 55})
B1_P1  :  Counter({0: 58060, 1: 15})
B1_P14  :  Counter({0: 57872, 1: 203})


In [457]:
prod_list_b2 = get_unique_values_from_list_col(df1, 'Product_Holding_B2')
prod_list_b2 = [item for item in prod_list_b2 if str(item)!='nan']
b2_prod_list = ['B2_'+item for item in prod_list_b2 ]
 
print(b2_prod_list)

print(len(b2_prod_list))
    
for item in b2_prod_list:
    print(item)
#     print(df1[item])
    df1[item]= 0 
    
# p21, p19 : not there    

['B2_P8', 'B2_P3', 'B2_P00', 'B2_P6', 'B2_P12', 'B2_P16', 'B2_P1', 'B2_P9', 'B2_P10', 'B2_P13', 'B2_P4', 'B2_P5', 'B2_P7', 'B2_P11', 'B2_P2', 'B2_P15', 'B2_P17', 'B2_P14', 'B2_P20', 'B2_P18']
20
B2_P8
B2_P3
B2_P00
B2_P6
B2_P12
B2_P16
B2_P1
B2_P9
B2_P10
B2_P13
B2_P4
B2_P5
B2_P7
B2_P11
B2_P2
B2_P15
B2_P17
B2_P14
B2_P20
B2_P18


In [458]:
df1.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,is_train,...,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
0,CC264719,1,41,14,0,1,3,[P16],[P8],1,...,0,0,0,0,0,0,0,0,0,0
1,CC209679,0,47,14,1,1,2,"[P13, P20]",[P3],1,...,0,0,0,0,0,0,0,0,0,0
2,CC319633,0,59,14,0,2,2,[P11],[P00],1,...,0,0,0,0,0,0,0,0,0,0
3,CC231413,0,32,16,0,1,2,"[P8, P13]",[P6],1,...,0,0,0,0,0,0,0,0,0,0
4,CC259633,1,30,15,0,2,3,"[P16, P17, P21]","[P8, P12]",1,...,0,0,0,0,0,0,0,0,0,0


In [459]:
df.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,is_train,...,B1_P18,B1_P15,B1_P6,B1_P9,B1_P7,B1_P3,B1_P5,B1_P4,B1_P1,B1_P14
0,CC264719,1,41,14,0,1,3,[P16],[P8],1,...,0,0,0,0,0,0,0,0,0,0
1,CC209679,0,47,14,1,1,2,"[P13, P20]",[P3],1,...,0,0,0,0,0,0,0,0,0,0
2,CC319633,0,59,14,0,2,2,[P11],[P00],1,...,0,0,0,0,0,0,0,0,0,0
3,CC231413,0,32,16,0,1,2,"[P8, P13]",[P6],1,...,0,0,0,0,0,0,0,0,0,0
4,CC259633,1,30,15,0,2,3,"[P16, P17, P21]","[P8, P12]",1,...,0,0,0,0,0,0,0,0,0,0


In [460]:
df.shape, df1.shape

((58075, 32), (58075, 52))

In [461]:
df_cols_details = list(df.columns)


df1_cols_details = list(df1.columns)


print(df.columns)

print(df1.columns)

Index(['Customer_ID', 'Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category',
       'Customer_Category', 'Product_Holding_B1', 'Product_Holding_B2',
       'is_train', 'B1_P16', 'B1_P13', 'B1_P20', 'B1_P11', 'B1_P8', 'B1_P17',
       'B1_P21', 'B1_P12', 'B1_P10', 'B1_P19', 'B1_P2', 'B1_P00', 'B1_P18',
       'B1_P15', 'B1_P6', 'B1_P9', 'B1_P7', 'B1_P3', 'B1_P5', 'B1_P4', 'B1_P1',
       'B1_P14'],
      dtype='object')
Index(['Customer_ID', 'Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category',
       'Customer_Category', 'Product_Holding_B1', 'Product_Holding_B2',
       'is_train', 'B1_P16', 'B1_P13', 'B1_P20', 'B1_P11', 'B1_P8', 'B1_P17',
       'B1_P21', 'B1_P12', 'B1_P10', 'B1_P19', 'B1_P2', 'B1_P00', 'B1_P18',
       'B1_P15', 'B1_P6', 'B1_P9', 'B1_P7', 'B1_P3', 'B1_P5', 'B1_P4', 'B1_P1',
       'B1_P14', 'B2_P8', 'B2_P3', 'B2_P00', 'B2_P6', 'B2_P12', 'B2_P16',
       'B2_P1', 'B2_P9', 'B2_P10', 'B2_P13', 'B2_P4', 'B2_P5', 'B2_P7',
       'B2_P11', 'B2_P2', 'B2_P15', 'B2_P1

In [462]:
train_df.shape

(37748, 10)

In [463]:
print(df_cols_details[9:])

print("\n*"*2)

print(df1_cols_details[9:])

['is_train', 'B1_P16', 'B1_P13', 'B1_P20', 'B1_P11', 'B1_P8', 'B1_P17', 'B1_P21', 'B1_P12', 'B1_P10', 'B1_P19', 'B1_P2', 'B1_P00', 'B1_P18', 'B1_P15', 'B1_P6', 'B1_P9', 'B1_P7', 'B1_P3', 'B1_P5', 'B1_P4', 'B1_P1', 'B1_P14']

*
*
['is_train', 'B1_P16', 'B1_P13', 'B1_P20', 'B1_P11', 'B1_P8', 'B1_P17', 'B1_P21', 'B1_P12', 'B1_P10', 'B1_P19', 'B1_P2', 'B1_P00', 'B1_P18', 'B1_P15', 'B1_P6', 'B1_P9', 'B1_P7', 'B1_P3', 'B1_P5', 'B1_P4', 'B1_P1', 'B1_P14', 'B2_P8', 'B2_P3', 'B2_P00', 'B2_P6', 'B2_P12', 'B2_P16', 'B2_P1', 'B2_P9', 'B2_P10', 'B2_P13', 'B2_P4', 'B2_P5', 'B2_P7', 'B2_P11', 'B2_P2', 'B2_P15', 'B2_P17', 'B2_P14', 'B2_P20', 'B2_P18']


In [464]:
df2 = df1.apply(apply_transformation, col='Product_Holding_B2',prod_list=prod_list_b2, prefix='B2_', axis=1)

print(df2.shape)


df2.head()

(58075, 52)


,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2,is_train,...,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
0,CC264719,1,41,14,0,1,3,[P16],[P8],1,...,0,0,0,0,0,0,0,0,0,0
1,CC209679,0,47,14,1,1,2,"[P13, P20]",[P3],1,...,0,0,0,0,0,0,0,0,0,0
2,CC319633,0,59,14,0,2,2,[P11],[P00],1,...,0,0,0,0,0,0,0,0,0,0
3,CC231413,0,32,16,0,1,2,"[P8, P13]",[P6],1,...,0,0,0,0,0,0,0,0,0,0
4,CC259633,1,30,15,0,2,3,"[P16, P17, P21]","[P8, P12]",1,...,0,0,0,0,0,0,0,0,0,0


In [465]:
df2['Product_Holding_B1'].apply(lambda x: len(x))

0        1
1        2
2        1
3        2
4        3
        ..
20322    1
20323    1
20324    1
20325    3
20326    3
Name: Product_Holding_B1, Length: 58075, dtype: int64

In [466]:
df3 = df2.copy()
df3['current_holdings'] = df3['Product_Holding_B1'].apply(lambda x: len(x))


df4 = df3.drop(columns = ['Product_Holding_B1','Product_Holding_B2'])

In [467]:
df4[df4['is_train'] == 1].to_csv('after_eda_processed_data/train_data.csv', index=False)


df4[df4['is_train'] == 0].to_csv('after_eda_processed_data/test_data.csv', index=False)

In [468]:
!pwd

/home/rakesh/Documents/DataScience/Hackathon/AmExpert 2021-ML


In [469]:
df_train = pd.read_csv('after_eda_processed_data/train_data.csv')
df_test = pd.read_csv('after_eda_processed_data/test_data.csv')

In [470]:
df_train.shape, df_test.shape

((37748, 51), (20327, 51))

In [471]:
df_train.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,is_train,B1_P16,B1_P13,...,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18,current_holdings
0,CC264719,1,41,14,0,1,3,1,1,0,...,0,0,0,0,0,0,0,0,0,1
1,CC209679,0,47,14,1,1,2,1,0,1,...,0,0,0,0,0,0,0,0,0,2
2,CC319633,0,59,14,0,2,2,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,CC231413,0,32,16,0,1,2,1,0,1,...,0,0,0,0,0,0,0,0,0,2
4,CC259633,1,30,15,0,2,3,1,1,0,...,0,0,0,0,0,0,0,0,0,3


In [472]:
df_train.columns

Index(['Customer_ID', 'Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category',
       'Customer_Category', 'is_train', 'B1_P16', 'B1_P13', 'B1_P20', 'B1_P11',
       'B1_P8', 'B1_P17', 'B1_P21', 'B1_P12', 'B1_P10', 'B1_P19', 'B1_P2',
       'B1_P00', 'B1_P18', 'B1_P15', 'B1_P6', 'B1_P9', 'B1_P7', 'B1_P3',
       'B1_P5', 'B1_P4', 'B1_P1', 'B1_P14', 'B2_P8', 'B2_P3', 'B2_P00',
       'B2_P6', 'B2_P12', 'B2_P16', 'B2_P1', 'B2_P9', 'B2_P10', 'B2_P13',
       'B2_P4', 'B2_P5', 'B2_P7', 'B2_P11', 'B2_P2', 'B2_P15', 'B2_P17',
       'B2_P14', 'B2_P20', 'B2_P18', 'current_holdings'],
      dtype='object')

In [473]:
cat_cols = ['Gender', 'Is_Active', 'City_Category','Customer_Category']
cols_to_drop = ['Customer_ID','is_train']

In [474]:
for col in list(df_train.columns):
#     print(col)
    
    if col.startswith('B1'):
        print(col)

B1_P16
B1_P13
B1_P20
B1_P11
B1_P8
B1_P17
B1_P21
B1_P12
B1_P10
B1_P19
B1_P2
B1_P00
B1_P18
B1_P15
B1_P6
B1_P9
B1_P7
B1_P3
B1_P5
B1_P4
B1_P1
B1_P14


In [475]:
B1_cols = [col for col in list(df_train.columns) if col.startswith('B1')]
print(B1_cols)
print(len(B1_cols))

['B1_P16', 'B1_P13', 'B1_P20', 'B1_P11', 'B1_P8', 'B1_P17', 'B1_P21', 'B1_P12', 'B1_P10', 'B1_P19', 'B1_P2', 'B1_P00', 'B1_P18', 'B1_P15', 'B1_P6', 'B1_P9', 'B1_P7', 'B1_P3', 'B1_P5', 'B1_P4', 'B1_P1', 'B1_P14']
22


In [476]:
B2_cols = [col for col in list(df_train.columns) if col.startswith('B2')]
print(B2_cols)
print(len(B2_cols))

['B2_P8', 'B2_P3', 'B2_P00', 'B2_P6', 'B2_P12', 'B2_P16', 'B2_P1', 'B2_P9', 'B2_P10', 'B2_P13', 'B2_P4', 'B2_P5', 'B2_P7', 'B2_P11', 'B2_P2', 'B2_P15', 'B2_P17', 'B2_P14', 'B2_P20', 'B2_P18']
20


In [477]:
cat_cols = B1_cols + cat_cols
print(cat_cols)
target_cols = B2_cols

['B1_P16', 'B1_P13', 'B1_P20', 'B1_P11', 'B1_P8', 'B1_P17', 'B1_P21', 'B1_P12', 'B1_P10', 'B1_P19', 'B1_P2', 'B1_P00', 'B1_P18', 'B1_P15', 'B1_P6', 'B1_P9', 'B1_P7', 'B1_P3', 'B1_P5', 'B1_P4', 'B1_P1', 'B1_P14', 'Gender', 'Is_Active', 'City_Category', 'Customer_Category']


In [478]:
df_train = df_train.drop(columns = cols_to_drop)

df_test = df_test.drop(columns = cols_to_drop)

In [479]:
X= df_train.drop(columns = target_cols)
y = df_train[target_cols]

In [480]:
X.shape , y.shape

((37748, 29), (37748, 20))

In [481]:
X.head()

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,B1_P16,B1_P13,B1_P20,B1_P11,...,B1_P15,B1_P6,B1_P9,B1_P7,B1_P3,B1_P5,B1_P4,B1_P1,B1_P14,current_holdings
0,1,41,14,0,1,3,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,47,14,1,1,2,0,1,1,0,...,0,0,0,0,0,0,0,0,0,2
2,0,59,14,0,2,2,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0,32,16,0,1,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
4,1,30,15,0,2,3,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [482]:
y.head()

,B2_P8,B2_P3,B2_P00,B2_P6,B2_P12,B2_P16,B2_P1,B2_P9,B2_P10,B2_P13,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [483]:
for cols in y.columns:
    print(cols)
    print(y[cols].unique())

B2_P8
[1 0]
B2_P3
[0 1]
B2_P00
[0 1]
B2_P6
[0 1]
B2_P12
[0 1]
B2_P16
[0 1]
B2_P1
[0 1]
B2_P9
[0 1]
B2_P10
[0 1]
B2_P13
[0 1]
B2_P4
[0 1]
B2_P5
[0 1]
B2_P7
[0 1]
B2_P11
[0 1]
B2_P2
[0 1]
B2_P15
[0 1]
B2_P17
[0 1]
B2_P14
[0 1]
B2_P20
[0 1]
B2_P18
[0 1]


In [511]:
x_train, x_test , y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=10)

In [512]:
from sklearn.multioutput import MultiOutputClassifier
import catboost
classifier = MultiOutputClassifier(catboost.CatBoostClassifier(verbose=1))

In [513]:
#!pip install catboost

In [514]:
# train_one_fold_x.columns

In [515]:
# x_train1, x_test1, y_train1, y_test1 = train_test_split(X ,y , test_size = 0.05 , random_state = 10)

# # create dataset for lightgbm
# lgb_train = lgb.Dataset(x_train1, y_train1)
# lgb_eval = lgb.Dataset(x_test1, y_test1, reference=lgb_train)

# # specify your configurations as a dict
# params = {
#     'boosting_type': 'gbdt',
    
#     'metric': {'l2', 'l1'},
#     'num_leaves': 31,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 0
# }

# print('Starting training...')
# # train
# classifier = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=20,
#                 valid_sets=lgb_eval,
#                 early_stopping_rounds=5)


# y_pred_final = gbm.predict(df4_test, num_iteration=gbm.best_iteration)


In [516]:
# classifier.fit(x_train, y_train, cat_features=cat_cols)


classifier.fit(x_train, y_train)

Learning rate set to 0.047506
0:	learn: 0.6657613	total: 17.8ms	remaining: 17.8s
1:	learn: 0.6387504	total: 27.2ms	remaining: 13.6s
2:	learn: 0.6166684	total: 34ms	remaining: 11.3s
3:	learn: 0.5963290	total: 40ms	remaining: 9.95s
4:	learn: 0.5787378	total: 45.9ms	remaining: 9.12s
5:	learn: 0.5673313	total: 51.4ms	remaining: 8.51s
6:	learn: 0.5570525	total: 56.6ms	remaining: 8.03s
7:	learn: 0.5477832	total: 62.1ms	remaining: 7.7s
8:	learn: 0.5370023	total: 67.8ms	remaining: 7.47s
9:	learn: 0.5286974	total: 73.2ms	remaining: 7.25s
10:	learn: 0.5202469	total: 78.9ms	remaining: 7.09s
11:	learn: 0.5144628	total: 84.4ms	remaining: 6.95s
12:	learn: 0.5099835	total: 89.5ms	remaining: 6.8s
13:	learn: 0.5057728	total: 95ms	remaining: 6.69s
14:	learn: 0.5024769	total: 100ms	remaining: 6.58s
15:	learn: 0.4979316	total: 106ms	remaining: 6.51s
16:	learn: 0.4951686	total: 112ms	remaining: 6.45s
17:	learn: 0.4906519	total: 117ms	remaining: 6.38s
18:	learn: 0.4883458	total: 122ms	remaining: 6.31s
19:	l

164:	learn: 0.4269047	total: 978ms	remaining: 4.95s
165:	learn: 0.4267834	total: 991ms	remaining: 4.98s
166:	learn: 0.4267198	total: 1s	remaining: 5.02s
167:	learn: 0.4264185	total: 1.01s	remaining: 5.02s
168:	learn: 0.4263800	total: 1.02s	remaining: 5.04s
169:	learn: 0.4262871	total: 1.04s	remaining: 5.06s
170:	learn: 0.4262466	total: 1.04s	remaining: 5.07s
171:	learn: 0.4261370	total: 1.05s	remaining: 5.08s
172:	learn: 0.4260861	total: 1.06s	remaining: 5.08s
173:	learn: 0.4260465	total: 1.07s	remaining: 5.08s
174:	learn: 0.4259950	total: 1.08s	remaining: 5.08s
175:	learn: 0.4259250	total: 1.08s	remaining: 5.08s
176:	learn: 0.4258969	total: 1.09s	remaining: 5.08s
177:	learn: 0.4257728	total: 1.1s	remaining: 5.08s
178:	learn: 0.4256109	total: 1.1s	remaining: 5.07s
179:	learn: 0.4254894	total: 1.11s	remaining: 5.07s
180:	learn: 0.4253827	total: 1.12s	remaining: 5.07s
181:	learn: 0.4253518	total: 1.13s	remaining: 5.09s
182:	learn: 0.4253120	total: 1.14s	remaining: 5.09s
183:	learn: 0.425

336:	learn: 0.4151534	total: 2.19s	remaining: 4.31s
337:	learn: 0.4151102	total: 2.2s	remaining: 4.31s
338:	learn: 0.4150681	total: 2.22s	remaining: 4.33s
339:	learn: 0.4149920	total: 2.24s	remaining: 4.35s
340:	learn: 0.4148989	total: 2.25s	remaining: 4.36s
341:	learn: 0.4148609	total: 2.27s	remaining: 4.37s
342:	learn: 0.4148179	total: 2.29s	remaining: 4.38s
343:	learn: 0.4147765	total: 2.31s	remaining: 4.4s
344:	learn: 0.4147220	total: 2.33s	remaining: 4.43s
345:	learn: 0.4146508	total: 2.35s	remaining: 4.43s
346:	learn: 0.4145799	total: 2.36s	remaining: 4.45s
347:	learn: 0.4145572	total: 2.37s	remaining: 4.44s
348:	learn: 0.4144865	total: 2.39s	remaining: 4.46s
349:	learn: 0.4144588	total: 2.4s	remaining: 4.46s
350:	learn: 0.4143960	total: 2.42s	remaining: 4.47s
351:	learn: 0.4143306	total: 2.43s	remaining: 4.48s
352:	learn: 0.4142997	total: 2.44s	remaining: 4.48s
353:	learn: 0.4142132	total: 2.46s	remaining: 4.48s
354:	learn: 0.4141548	total: 2.46s	remaining: 4.48s
355:	learn: 0.4

501:	learn: 0.4076871	total: 3.62s	remaining: 3.59s
502:	learn: 0.4076297	total: 3.63s	remaining: 3.58s
503:	learn: 0.4075755	total: 3.63s	remaining: 3.58s
504:	learn: 0.4075595	total: 3.64s	remaining: 3.57s
505:	learn: 0.4075408	total: 3.65s	remaining: 3.56s
506:	learn: 0.4075186	total: 3.65s	remaining: 3.55s
507:	learn: 0.4074677	total: 3.66s	remaining: 3.55s
508:	learn: 0.4074532	total: 3.67s	remaining: 3.54s
509:	learn: 0.4074179	total: 3.67s	remaining: 3.53s
510:	learn: 0.4073801	total: 3.68s	remaining: 3.52s
511:	learn: 0.4073555	total: 3.68s	remaining: 3.51s
512:	learn: 0.4073246	total: 3.69s	remaining: 3.5s
513:	learn: 0.4072819	total: 3.7s	remaining: 3.5s
514:	learn: 0.4071965	total: 3.7s	remaining: 3.49s
515:	learn: 0.4071613	total: 3.71s	remaining: 3.48s
516:	learn: 0.4071306	total: 3.72s	remaining: 3.47s
517:	learn: 0.4071072	total: 3.72s	remaining: 3.46s
518:	learn: 0.4070623	total: 3.73s	remaining: 3.46s
519:	learn: 0.4070379	total: 3.73s	remaining: 3.45s
520:	learn: 0.40

662:	learn: 0.4013170	total: 4.62s	remaining: 2.35s
663:	learn: 0.4012462	total: 4.63s	remaining: 2.34s
664:	learn: 0.4012024	total: 4.63s	remaining: 2.33s
665:	learn: 0.4011794	total: 4.64s	remaining: 2.33s
666:	learn: 0.4011485	total: 4.64s	remaining: 2.32s
667:	learn: 0.4011112	total: 4.65s	remaining: 2.31s
668:	learn: 0.4010855	total: 4.66s	remaining: 2.3s
669:	learn: 0.4010443	total: 4.66s	remaining: 2.29s
670:	learn: 0.4009857	total: 4.67s	remaining: 2.29s
671:	learn: 0.4009250	total: 4.67s	remaining: 2.28s
672:	learn: 0.4008714	total: 4.68s	remaining: 2.27s
673:	learn: 0.4008378	total: 4.68s	remaining: 2.27s
674:	learn: 0.4007907	total: 4.69s	remaining: 2.26s
675:	learn: 0.4007738	total: 4.69s	remaining: 2.25s
676:	learn: 0.4007477	total: 4.7s	remaining: 2.24s
677:	learn: 0.4007351	total: 4.7s	remaining: 2.23s
678:	learn: 0.4006974	total: 4.71s	remaining: 2.23s
679:	learn: 0.4006548	total: 4.71s	remaining: 2.22s
680:	learn: 0.4006175	total: 4.72s	remaining: 2.21s
681:	learn: 0.4

...

349:	learn: 0.0000161	total: 1.98s	remaining: 3.67s
350:	learn: 0.0000160	total: 1.98s	remaining: 3.67s
351:	learn: 0.0000160	total: 1.99s	remaining: 3.66s
352:	learn: 0.0000159	total: 2s	remaining: 3.66s
353:	learn: 0.0000159	total: 2s	remaining: 3.66s
354:	learn: 0.0000158	total: 2.01s	remaining: 3.65s
355:	learn: 0.0000157	total: 2.02s	remaining: 3.65s
356:	learn: 0.0000157	total: 2.02s	remaining: 3.64s
357:	learn: 0.0000156	total: 2.03s	remaining: 3.64s
358:	learn: 0.0000155	total: 2.03s	remaining: 3.63s
359:	learn: 0.0000155	total: 2.04s	remaining: 3.63s
360:	learn: 0.0000154	total: 2.05s	remaining: 3.62s
361:	learn: 0.0000154	total: 2.05s	remaining: 3.62s
362:	learn: 0.0000153	total: 2.06s	remaining: 3.61s
363:	learn: 0.0000153	total: 2.06s	remaining: 3.6s
364:	learn: 0.0000152	total: 2.07s	remaining: 3.59s
365:	learn: 0.0000152	total: 2.07s	remaining: 3.59s
366:	learn: 0.0000151	total: 2.08s	remaining: 3.58s
367:	learn: 0.0000151	total: 2.08s	remaining: 3.57s
368:	learn: 0.00001

537:	learn: 0.0000100	total: 2.96s	remaining: 2.55s
538:	learn: 0.0000100	total: 2.98s	remaining: 2.54s
539:	learn: 0.0000100	total: 2.98s	remaining: 2.54s
540:	learn: 0.0000099	total: 2.99s	remaining: 2.54s
541:	learn: 0.0000099	total: 2.99s	remaining: 2.53s
542:	learn: 0.0000099	total: 3s	remaining: 2.52s
543:	learn: 0.0000099	total: 3s	remaining: 2.52s
544:	learn: 0.0000099	total: 3.01s	remaining: 2.51s
545:	learn: 0.0000098	total: 3.01s	remaining: 2.51s
546:	learn: 0.0000098	total: 3.02s	remaining: 2.5s
547:	learn: 0.0000098	total: 3.02s	remaining: 2.49s
548:	learn: 0.0000098	total: 3.03s	remaining: 2.49s
549:	learn: 0.0000097	total: 3.03s	remaining: 2.48s
550:	learn: 0.0000097	total: 3.04s	remaining: 2.48s
551:	learn: 0.0000097	total: 3.04s	remaining: 2.47s
552:	learn: 0.0000097	total: 3.05s	remaining: 2.46s
553:	learn: 0.0000097	total: 3.06s	remaining: 2.46s
554:	learn: 0.0000096	total: 3.06s	remaining: 2.46s
555:	learn: 0.0000096	total: 3.07s	remaining: 2.45s
556:	learn: 0.00000

726:	learn: 0.0000069	total: 3.95s	remaining: 1.48s
727:	learn: 0.0000069	total: 3.96s	remaining: 1.48s
728:	learn: 0.0000069	total: 3.97s	remaining: 1.47s
729:	learn: 0.0000069	total: 3.97s	remaining: 1.47s
730:	learn: 0.0000069	total: 3.98s	remaining: 1.46s
731:	learn: 0.0000069	total: 3.98s	remaining: 1.46s
732:	learn: 0.0000069	total: 3.99s	remaining: 1.45s
733:	learn: 0.0000069	total: 3.99s	remaining: 1.45s
734:	learn: 0.0000068	total: 4s	remaining: 1.44s
735:	learn: 0.0000068	total: 4s	remaining: 1.44s
736:	learn: 0.0000068	total: 4.01s	remaining: 1.43s
737:	learn: 0.0000068	total: 4.01s	remaining: 1.42s
738:	learn: 0.0000068	total: 4.02s	remaining: 1.42s
739:	learn: 0.0000068	total: 4.02s	remaining: 1.41s
740:	learn: 0.0000068	total: 4.03s	remaining: 1.41s
741:	learn: 0.0000068	total: 4.03s	remaining: 1.4s
742:	learn: 0.0000068	total: 4.04s	remaining: 1.4s
743:	learn: 0.0000067	total: 4.04s	remaining: 1.39s
744:	learn: 0.0000067	total: 4.05s	remaining: 1.39s
745:	learn: 0.000006

912:	learn: 0.0000052	total: 4.95s	remaining: 472ms
913:	learn: 0.0000052	total: 4.96s	remaining: 466ms
914:	learn: 0.0000052	total: 4.96s	remaining: 461ms
915:	learn: 0.0000052	total: 4.97s	remaining: 456ms
916:	learn: 0.0000052	total: 4.97s	remaining: 450ms
917:	learn: 0.0000052	total: 4.98s	remaining: 445ms
918:	learn: 0.0000052	total: 4.99s	remaining: 439ms
919:	learn: 0.0000052	total: 4.99s	remaining: 434ms
920:	learn: 0.0000052	total: 5s	remaining: 429ms
921:	learn: 0.0000052	total: 5s	remaining: 423ms
922:	learn: 0.0000052	total: 5.01s	remaining: 418ms
923:	learn: 0.0000052	total: 5.01s	remaining: 412ms
924:	learn: 0.0000051	total: 5.02s	remaining: 407ms
925:	learn: 0.0000051	total: 5.02s	remaining: 401ms
926:	learn: 0.0000051	total: 5.03s	remaining: 396ms
927:	learn: 0.0000051	total: 5.03s	remaining: 390ms
928:	learn: 0.0000051	total: 5.04s	remaining: 385ms
929:	learn: 0.0000051	total: 5.04s	remaining: 379ms
930:	learn: 0.0000051	total: 5.05s	remaining: 374ms
931:	learn: 0.0000

MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7f901a672cd0>)

In [518]:
print(classifier.score(x_test, y_test))

0.3103813559322034


In [519]:

preds = classifier.predict_proba(x_test)


preds_cal = []
for item in preds:
    preds_cal.append( [local_item[1] for local_item in item] )
    
    

In [520]:
preds_cal

[[0.00858280950999723,
  0.0008244776945687527,
  0.3485128500703819,
  0.5660050402105067,
  0.7067390984181868,
  0.37174638633536355,
  0.0630643360854644,
  0.0014060156766480605,
  0.0024692532592330064,
  0.68997662215225,
  0.23296358205911927,
  0.3417716999654741,
  0.8640185637196853,
  0.9260945947127625,
  0.14944385314141853,
  0.315066029080324,
  0.2646447884351549,
  0.9173188904823416,
  0.002696020524506192,
  0.35521780158421373,
  0.315693695715743,
  0.44071958108445497,
  0.23118286031240787,
  0.27435137945271426,
  0.09800365522796883,
  0.9363668876850628,
  0.29861259808401597,
  0.0005960439213618292,
  0.015322493012445275,
  0.16973396411827466,
  0.696943769393303,
  0.016860300138344254,
  0.51508830410057,
  0.30964325018864064,
  0.4996297365280945,
  0.0005721654920682041,
  0.42691547612409825,
  0.21939840605540356,
  0.0008119692428294419,
  0.2990893987948274,
  0.29274736678310137,
  0.5130183894159213,
  0.9585489234914455,
  0.000387091667917933

In [521]:
df_catboost = pd.DataFrame(np.array(preds_cal).T, columns=B2_cols)

In [522]:
df_catboost

,B2_P8,B2_P3,B2_P00,B2_P6,B2_P12,B2_P16,B2_P1,B2_P9,B2_P10,B2_P13,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
0,0.008583,0.011699,6.499764e-07,0.017079,0.007033,6.581384e-06,0.243040,0.006435,0.015725,9.275378e-06,0.619143,0.010861,0.015339,1.007829e-06,5.902524e-04,1.041024e-03,9.108112e-07,2.361115e-06,2.090965e-06,4.794656e-08
1,0.000824,0.044029,3.403133e-06,0.655677,0.001880,1.533998e-06,0.028145,0.000069,0.000421,5.791564e-07,0.041835,0.011428,0.164524,2.224882e-05,5.603646e-04,8.993477e-07,2.532552e-07,1.082832e-05,2.008741e-06,3.091836e-07
2,0.348513,0.001559,7.113920e-02,0.139534,0.055302,2.914818e-05,0.002312,0.001014,0.190787,2.122624e-05,0.099905,0.035689,0.047339,2.445724e-07,1.351416e-06,1.684964e-04,5.568654e-07,3.016590e-07,5.139678e-07,6.176609e-08
3,0.566005,0.003914,2.944843e-02,0.144580,0.073821,3.323184e-05,0.004774,0.002722,0.074713,3.263366e-05,0.018443,0.018205,0.035221,8.477403e-06,2.369600e-05,2.536909e-03,5.314535e-08,2.125773e-06,9.857427e-07,2.901404e-07
4,0.706739,0.093730,1.997917e-06,0.089434,0.842468,1.052787e-04,0.070415,0.147416,0.145552,5.202397e-05,0.210891,0.034508,0.089809,2.282792e-07,2.244744e-07,8.629978e-03,6.756216e-04,2.038841e-06,2.849069e-06,1.207753e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,0.593842,0.002671,4.436271e-02,0.105309,0.189244,5.545271e-05,0.001948,0.003499,0.025166,1.108812e-05,0.016311,0.007768,0.026706,1.848551e-05,2.572936e-05,2.337084e-04,1.581368e-07,1.100014e-06,6.868459e-07,4.137509e-08
1884,0.377768,0.031900,1.330345e-06,0.038684,0.000094,1.300040e-07,0.033687,0.052466,0.515175,1.240520e-06,0.113363,0.052918,0.010816,1.814747e-05,4.228326e-05,2.069487e-07,1.366186e-07,5.791023e-09,2.322978e-06,1.665355e-07
1885,0.264683,0.011934,5.528617e-07,0.030375,0.934007,8.722759e-05,0.009987,0.007085,0.010943,4.431823e-05,0.010038,0.004835,0.010752,1.388839e-05,2.940010e-05,1.641294e-04,1.300686e-08,5.863026e-05,2.123680e-06,1.516463e-07
1886,0.446601,0.002607,4.523723e-02,0.191562,0.239023,5.482597e-05,0.003878,0.000986,0.025669,2.245359e-05,0.024147,0.010417,0.055023,2.423521e-06,5.554767e-06,6.222425e-03,6.320639e-07,3.025782e-06,5.229368e-07,8.738120e-08


In [523]:
df_catboost.to_csv('results/catboost_preds.csv', index=False)

In [524]:
df_test_45 = pd.read_csv('test.csv')

df_cat = pd.read_csv('results/catboost_preds.csv')

df_test_45.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1
0,CC372708,Female,31,31,0,C2,S3,"['P12', 'P13']"
1,CC216072,Male,28,37,1,C1,S2,"['P12', 'P13']"
2,CC387629,Male,31,12,0,C2,S3,['P20']
3,CC389228,Female,55,11,0,C2,S2,"['P13', 'P21']"
4,CC394445,Male,51,49,1,C2,S1,['P13']


In [525]:
df_cat.head()

,B2_P8,B2_P3,B2_P00,B2_P6,B2_P12,B2_P16,B2_P1,B2_P9,B2_P10,B2_P13,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
0,0.008583,0.011699,6.499764e-07,0.017079,0.007033,0.000007,0.243040,0.006435,0.015725,9.275378e-06,0.619143,0.010861,0.015339,1.007829e-06,5.902524e-04,1.041024e-03,9.108112e-07,2.361115e-06,2.090965e-06,4.794656e-08
1,0.000824,0.044029,3.403133e-06,0.655677,0.001880,0.000002,0.028145,0.000069,0.000421,5.791564e-07,0.041835,0.011428,0.164524,2.224882e-05,5.603646e-04,8.993477e-07,2.532552e-07,1.082832e-05,2.008741e-06,3.091836e-07
2,0.348513,0.001559,7.113920e-02,0.139534,0.055302,0.000029,0.002312,0.001014,0.190787,2.122624e-05,0.099905,0.035689,0.047339,2.445724e-07,1.351416e-06,1.684964e-04,5.568654e-07,3.016590e-07,5.139678e-07,6.176609e-08
3,0.566005,0.003914,2.944843e-02,0.144580,0.073821,0.000033,0.004774,0.002722,0.074713,3.263366e-05,0.018443,0.018205,0.035221,8.477403e-06,2.369600e-05,2.536909e-03,5.314535e-08,2.125773e-06,9.857427e-07,2.901404e-07
4,0.706739,0.093730,1.997917e-06,0.089434,0.842468,0.000105,0.070415,0.147416,0.145552,5.202397e-05,0.210891,0.034508,0.089809,2.282792e-07,2.244744e-07,8.629978e-03,6.756216e-04,2.038841e-06,2.849069e-06,1.207753e-05


In [526]:
df_test_45.shape, df_cat.shape

((20327, 8), (1888, 20))

In [527]:
df_45 = pd.concat([df_test_45, df_cat], axis=1)
df_45.shape

(20327, 28)

In [528]:
df_45.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,B2_P8,B2_P3,...,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
0,CC372708,Female,31,31,0,C2,S3,"['P12', 'P13']",0.008583,0.011699,...,0.619143,0.010861,0.015339,1.007829e-06,5.902524e-04,1.041024e-03,9.108112e-07,2.361115e-06,2.090965e-06,4.794656e-08
1,CC216072,Male,28,37,1,C1,S2,"['P12', 'P13']",0.000824,0.044029,...,0.041835,0.011428,0.164524,2.224882e-05,5.603646e-04,8.993477e-07,2.532552e-07,1.082832e-05,2.008741e-06,3.091836e-07
2,CC387629,Male,31,12,0,C2,S3,['P20'],0.348513,0.001559,...,0.099905,0.035689,0.047339,2.445724e-07,1.351416e-06,1.684964e-04,5.568654e-07,3.016590e-07,5.139678e-07,6.176609e-08
3,CC389228,Female,55,11,0,C2,S2,"['P13', 'P21']",0.566005,0.003914,...,0.018443,0.018205,0.035221,8.477403e-06,2.369600e-05,2.536909e-03,5.314535e-08,2.125773e-06,9.857427e-07,2.901404e-07
4,CC394445,Male,51,49,1,C2,S1,['P13'],0.706739,0.093730,...,0.210891,0.034508,0.089809,2.282792e-07,2.244744e-07,8.629978e-03,6.756216e-04,2.038841e-06,2.849069e-06,1.207753e-05


In [529]:
df_45.tail()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,B2_P8,B2_P3,...,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
20322,CC303542,Female,37,11,1,C2,S3,['P16'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20323,CC266713,Male,29,14,0,C2,S2,['P16'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20324,CC393639,Male,26,12,0,C2,S3,['P13'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20325,CC285013,Female,44,19,1,C1,S3,"['P8', 'P9', 'P16']",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20326,CC274657,Female,25,21,0,C1,S3,"['P12', 'P13', 'P17']",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [530]:
total_cols = [col for col in df_45.columns if col.startswith('B2_')]
# total_cols

In [531]:
def get_labels(row):
    labels = []
    max_val = -1
    max_label = ''
    label_dict = {}
    for col in total_cols:
        if row[col] > max_val:
            max_val = row[col]
            max_label = col.split('_')[-1] 

        if row[col] > 0.5:
            labels.append(col.split('_')[-1])
            label_dict[col.split('_')[-1]] = row[col]
    
    if len(labels) == 0:
        labels.append(max_label)
        
    if len(labels) > 3:
        sorted_dict = dict(sorted(label_dict.items(), key=lambda item: item[1], reverse=True))
        print(sorted_dict)
        labels = []
        labels.extend(list(sorted_dict.keys())[0:3])
        
    return str(labels)

In [532]:
df_46 = df_45.copy()

In [533]:
df_46['Product_Holding_B2'] = df_46.apply( get_labels, axis=1 )

{'P8': 0.9585489234914456, 'P5': 0.8686323117664047, 'P1': 0.5538553064032448, 'P4': 0.5290495481327949}
{'P8': 0.9695018043356748, 'P4': 0.8123650012846382, 'P3': 0.5892246998393131, 'P7': 0.5619579486893757}
{'P12': 0.9738442823951784, 'P8': 0.8398825930963383, 'P10': 0.8015013677235144, 'P4': 0.7595607096433779, 'P9': 0.5001902020313228}
{'P8': 0.9242924757436334, 'P4': 0.8789561185086994, 'P1': 0.74042843726381, 'P6': 0.7128709862748714, 'P9': 0.6301886877103123}
{'P8': 0.9690584599227569, 'P4': 0.9673662067946908, 'P7': 0.7590073742962742, 'P6': 0.7256828793270447}
{'P8': 0.9428059011469516, 'P1': 0.6837031302996749, 'P7': 0.5626127013919316, 'P3': 0.5277955014357084}
{'P8': 0.9828416070587448, 'P4': 0.7791407585150628, 'P1': 0.6151199711177877, 'P7': 0.5956402074097147, 'P3': 0.5322845495001103}
{'P8': 0.9588457416394972, 'P12': 0.933874213308935, 'P6': 0.5949057832789688, 'P3': 0.5480634195391194, 'P10': 0.5335874212957973, 'P1': 0.5254361254493781}
{'P13': 0.9978671203079702, '

In [534]:
df_45.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,B2_P8,B2_P3,...,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
0,CC372708,Female,31,31,0,C2,S3,"['P12', 'P13']",0.008583,0.011699,...,0.619143,0.010861,0.015339,1.007829e-06,5.902524e-04,1.041024e-03,9.108112e-07,2.361115e-06,2.090965e-06,4.794656e-08
1,CC216072,Male,28,37,1,C1,S2,"['P12', 'P13']",0.000824,0.044029,...,0.041835,0.011428,0.164524,2.224882e-05,5.603646e-04,8.993477e-07,2.532552e-07,1.082832e-05,2.008741e-06,3.091836e-07
2,CC387629,Male,31,12,0,C2,S3,['P20'],0.348513,0.001559,...,0.099905,0.035689,0.047339,2.445724e-07,1.351416e-06,1.684964e-04,5.568654e-07,3.016590e-07,5.139678e-07,6.176609e-08
3,CC389228,Female,55,11,0,C2,S2,"['P13', 'P21']",0.566005,0.003914,...,0.018443,0.018205,0.035221,8.477403e-06,2.369600e-05,2.536909e-03,5.314535e-08,2.125773e-06,9.857427e-07,2.901404e-07
4,CC394445,Male,51,49,1,C2,S1,['P13'],0.706739,0.093730,...,0.210891,0.034508,0.089809,2.282792e-07,2.244744e-07,8.629978e-03,6.756216e-04,2.038841e-06,2.849069e-06,1.207753e-05


In [535]:
df_46.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,B2_P8,B2_P3,...,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18,Product_Holding_B2
0,CC372708,Female,31,31,0,C2,S3,"['P12', 'P13']",0.008583,0.011699,...,0.010861,0.015339,1.007829e-06,5.902524e-04,1.041024e-03,9.108112e-07,2.361115e-06,2.090965e-06,4.794656e-08,['P4']
1,CC216072,Male,28,37,1,C1,S2,"['P12', 'P13']",0.000824,0.044029,...,0.011428,0.164524,2.224882e-05,5.603646e-04,8.993477e-07,2.532552e-07,1.082832e-05,2.008741e-06,3.091836e-07,['P6']
2,CC387629,Male,31,12,0,C2,S3,['P20'],0.348513,0.001559,...,0.035689,0.047339,2.445724e-07,1.351416e-06,1.684964e-04,5.568654e-07,3.016590e-07,5.139678e-07,6.176609e-08,['P8']
3,CC389228,Female,55,11,0,C2,S2,"['P13', 'P21']",0.566005,0.003914,...,0.018205,0.035221,8.477403e-06,2.369600e-05,2.536909e-03,5.314535e-08,2.125773e-06,9.857427e-07,2.901404e-07,['P8']
4,CC394445,Male,51,49,1,C2,S1,['P13'],0.706739,0.093730,...,0.034508,0.089809,2.282792e-07,2.244744e-07,8.629978e-03,6.756216e-04,2.038841e-06,2.849069e-06,1.207753e-05,"['P8', 'P12']"


In [537]:
df_46[['Customer_ID', 'Product_Holding_B2']].to_csv('submission/submission_catboost_3.csv', index=False)